In [ ]:
import torch
from transformers import BertModel

# Load the pre-trained BERT model
model = BertModel.from_pretrained("bert-base-uncased")
model.to("cuda")

In [ ]:
torch._dynamo.reset()
model_compiled = torch.compile(
        model,
        options={
            "trace.enabled": True,
        },
)

In [ ]:
EPOCHS = 10
LEARNING_RATE = 1e-7
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
input_tensor = model.dummy_inputs["input_ids"].clone().to("cuda")
labels = input_tensor.clone().to("cuda")
targets = input_tensor.clone().to("cuda")

model_compiled.train()

for epoch in range(2):
    optimizer.zero_grad()
    outputs = model_compiled(input_tensor, labels)
    loss = loss_function(outputs.pooler_output, outputs.pooler_output)
    loss.backward()
    optimizer.step()

# Save the pre-trained model
# model.save_pretrained("my_bert_model")